In [58]:
import open3d as o3d
import numpy as np
import pykitti
import os
import matplotlib.pyplot as plt
import torch

In [56]:
path = "/media/sandhu/SSD/dataset/kitti/odometry/dataset/sequences/08/velodyne/"
bin_file_path = sorted(os.listdir(path))

full_file_path = [os.path.join(path, i) for i in bin_file_path]

def bin2array(file):
    array = np.fromfile(file, dtype=np.float32).reshape(-1, 4)
    points = array[:, :3]
    intensity = array[:, -1]
    return points, intensity

def points_to_homo(points):
    ones = np.ones(len(points)).reshape(-1,1)
    points = np.hstack((points, ones))
    return points

def cvt_frame(gt_poses, idx, points):
    points_w = gt_poses[idx] @ points.T
    points_w = np.delete(points_w.T, -1, axis=1)
    return points_w

def get_gt_poses():
    """This function loads the GT poses on the Velodyne coordinate frame."""
    data = pykitti.odometry("/media/sandhu/SSD/dataset/kitti/odometry/dataset", "08")
    T_cam_velo = data.calib.T_cam0_velo
    T_velo_cam = np.linalg.inv(T_cam_velo)

    gt_poses = T_velo_cam @ data.poses @ T_cam_velo
    return gt_poses

def points2pcd(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

In [57]:
gt_poses = get_gt_poses()

for idx, i in enumerate(full_file_path):
    points, _ = bin2array(i)
    points = points_to_homo(points)
    points = cvt_frame(gt_poses, idx, points)
    pcd = points2pcd(points)
    o3d.io.write_point_cloud(f"pcd/{idx}.pcd", pcd)

In [79]:
xyz = np.array(torch.load("tools/xyz.pt").cpu())
print(xyz[: 10])

[[ 20.071732    10.954449    -1.1626467 ]
 [ 25.973503     0.24628448  -0.73464763]
 [ 17.740112   -19.655722    -0.3506791 ]
 [ 15.05806    -17.111658    -0.31844682]
 [ 17.940247   -13.328232    -0.5288076 ]
 [ 21.913956    -5.3029366   -0.46488124]
 [ 40.274864     8.531654    -0.77705824]
 [ 54.300148    -2.3400154   -0.539096  ]
 [ 35.233498    -5.727024    -0.17135626]
 [ 62.10038      3.905964    -0.43801457]]


In [85]:
# xyz = points_to_homo(xyz)
# cvt_xyz = cvt_frame(gt_poses, 0, xyz)
# print(cvt_xyz)
print(gt_poses[500])

[[ 5.22616036e-01  8.52458394e-01 -1.36804251e-02  7.88339501e+01]
 [-8.51979047e-01  5.22784140e-01  2.87854455e-02  3.78958391e+02]
 [ 3.16903152e-02 -3.38830075e-03  9.99491923e-01 -7.59785332e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
